In [1]:
import os
import glob
import random
import pandas as pd
import tiktoken
import datetime
from collections import Counter
from IPython.display import display, Markdown

# user DF에서 History column과 Train column을 합쳐서 history로 사용
user = pd.read_csv('../../data/user.tsv', sep='\t', names=['User', 'History', 'Train', 'Question'])
user['History'] = user['History'] + user['Train']
user = user.drop(columns=['Train'])

history_news = pd.read_csv('../../data/history/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
train_news = pd.read_csv('../../data/train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
history_news = pd.concat([history_news, train_news], ignore_index=True)
history_news[['Category_New', 'SubCategory']] = history_news['Category'].str.split('|', expand=True)
history_news = history_news.drop('Category', axis=1).rename(columns={'Category_New': 'Category'})

question_news = pd.read_csv('../../data/test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
question_news[['Category_New', 'SubCategory']] = question_news['Category'].str.split('|', expand=True)
question_news = question_news.drop('Category', axis=1).rename(columns={'Category_New': 'Category'})


# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
question_news_sorted = question_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

In [2]:
instruction = """
The news articles that User #2074 clicked before are as follows:
1. Se lesernes nyttårsbilder
2. Politiet frigir navn på kvinne som døde etter E39-ulykke
3. Nå er navnet frigitt etter dødsfall i Gjemnes
4. Dette har Rema 1000 søkt om å bruke Æ-varemerket til
5. - Helt ufattelig at det går an å behandle en ny bil på denne måten
6. Slik gikk det da elbilene måtte betale p-avgift
7. Boligselger må punge ut for å ha krysset av feil om oppussing
8. Tøft Rema-år er bakteppe for «Æ»
9. Reitan etablerer ny bensinstasjonkjede
10. Oppretter ikke straffesak etter denne utforkjøringen
11. Trafikkuhell i Elgesetergate
12. To vogntog har kollidert
13. Nå har Rema avslørt hva Æ er
14. Æ-en ble en nedtur
15. Koteng mener Reitan dekkes for negativt
16. - Vi skal ikke drive med biljakt
17. Nils Arne Eggen: - Akkurat nå bruker jeg gåstol og krykker
18. Ønsket seg utedo-feeling på badet

Based on the articles the user clicked before, User #2074 prefers most [MASK] among the following five articles:
1: Politiet har tre ulike teorier etter dødsfallet i Tydal
2: Flyktninger frøs i hjel i Bulgaria
3: Personbil krasjet i trailer etter forbikjøring
4: Nå skal disse bort fra Londons gater
5: Eksperter gir råd om Norges bidrag til NATOs rakettforsvar
Question 1. The index number of the [MASK] is 3

Based on the articles the user clicked before, User #2074 prefers most [MASK] among the following five articles:
1: Gigantisk isfjell holder på å brekke av i Antarktis
2: Nå endrer vi matvanene. Det får ekspertene til å juble
3: Uklart om flyplasskyting i Florida var terrorangrep
4: Selvskrytet ville nesten ingen ende ta. Trondhjemmerne var i skyene
5: Hva er vitsen med å bo i Norge hvis man hater snø?
Question 2. The index number of the [MASK] is 2

The news articles that User #2079 clicked before are as follows:
1. Se lesernes nyttårsbilder
2. Her koker det over for Tønseth. Så stakk han fra stadion i sinne.
3. Åpenbart beruset mann i trafikkulykke på Byåsen
4. Frontkollisjon på Heimdal
5. Slik bor Trondheims «Anno»-deltagere
6. To skadde etter dødsulykke på E39 får behandling ved St. Olav
7. Dette har Rema 1000 søkt om å bruke Æ-varemerket til
8. Politiet frigir navn på kvinne som døde etter E39-ulykke
9. Nå er navnet frigitt etter dødsfall i Gjemnes
10. - Helt ufattelig at det går an å behandle en ny bil på denne måten
11. - Dette er ikke Senterpartiets olje- og gasspolitikk
12. - Fast ansatte er et konkurransefortrinn
13. NHO vil fjerne mastergrader
14. Gikk du glipp av romjulssalget? Frykt ikke, tidenes januarsalg er her. Igjen
15. Da han kom tilbake fra alpinbakken så bilen slik ut
16. To vogntog har kollidert
17. Det får da være grenser for dårlig planlegging ved kjøp av nye togvogner
18. Æ-en ble en nedtur
19. En usedvanlig imponerende smartmobil
20. Trafikkuhell i Elgesetergate
21. Her er Samsungs nye super-TV-er
22. Lastebil mistet last med metallrør i veien
23. Æ-appen førte til betalingsproblemer i Rema-butikkene
24. Almhjell forlater sitt fantasy-univers
25. Stor test av 42 typer knekkebrød
26. Dette var starten for avisen som er eldre enn USA
27. Norsk Tipping deler ut 448 millioner. Se hvor mye din klubb får.
28. Historiske trondheimsbilder: Her tar Tyholttårnet form
29. En telefon uten knapper? Vi trodde knapt det var mulig
30. Kødannelse på E6 etter at bil mistet hjul
31. Nils Arne Eggen: - Akkurat nå bruker jeg gåstol og krykker

Based on the articles the user clicked before, User #2079 prefers most [MASK] among the following five articles:
1: Tande har funnet årsaken til fiaskohoppet: Tar på seg all skyld
2: Tror de lave rentene er blitt en sovepute for naive lånekunder
3: Tankestreif: - Jeg er da ikke helt idiot heller!
4: VM i Lahti kan gå uten en eneste Northug: – Akkurat nå ser det mørkt ut
5: 3T åpner to nye treningssenter
Question 1. The index number of the [MASK] is 5

Based on the articles the user clicked before, User #2079 prefers most [MASK] among the following five articles:
1: Gigantisk isfjell holder på å brekke av i Antarktis
2: Nå skal disse bort fra Londons gater
3: Svensk rapport: Russland forsøker å spre falske nyheter
4: - Det ser ikke ut som om det har skjedd noe straffbart
5: Politiet har tre ulike teorier etter dødsfallet i Tydal
Question 2. The index number of the [MASK] is 1

Based on the articles the user clicked before, User #2079 prefers most [MASK] among the following five articles:
1: Weng og Østberg parkert etter smørebom: – Dette var grusomt
2: Personbil krasjet i trailer etter forbikjøring
3: Selvskrytet ville nesten ingen ende ta. Trondhjemmerne var i skyene
4: Nå endrer vi matvanene. Det får ekspertene til å juble
5: Kantspiller klar for Rosenborg: – En god løsning
Question 3. The index number of the [MASK] is 3
"""

In [3]:
def extract_news_ids_and_click_times(news_str):
    """
    목적: 
    - 뉴스 click history 문자열에서 뉴스 ID와 클릭 시간을 추출하여 각각의 리스트로 반환 (중복된 뉴스 ID는 제외)

    설명:
    - <news_str>을 세미콜론으로 분리한 후, 각 항목에서 뉴스 ID와 클릭 시간을 추출
    - 중복된 뉴스 ID는 <seen_ids> 집합을 사용하여 제외
    - 클릭 시간은 pandas의 <to_datetime> 함수를 사용하여 datetime 객체로 변환됨
    """
    news_ids = []     # 뉴스 ID를 저장할 리스트
    click_times = []  # 클릭 시간을 저장할 리스트
    seen_ids = set()  # 중복된 뉴스 ID를 추적하기 위한 집합
    entries = news_str.strip(';').split(';')  # 세미콜론으로 구분된 항목들 분리
    for entry in entries:
        if entry:
            parts = entry.split(',')  # 쉼표로 구분된 부분들 분리
            if len(parts) >= 3:
                news_id = parts[0]  # 첫 번째 부분을 뉴스 ID로 사용
                click_time_str = parts[-1]  # 마지막 부분을 클릭 시간으로 사용
                click_time = pd.to_datetime(click_time_str)  # 문자열을 datetime으로 변환
                if news_id not in seen_ids:
                    seen_ids.add(news_id)  # 중복 방지를 위해 집합에 추가
                    news_ids.append(news_id)
                    click_times.append(click_time)
    return news_ids, click_times

def find_similar_news(news_df, news_ids, click_times, used_ids, used_titles, low_freq_categories):
    """
    목적: 
    - 질문 뉴스와 유사한 뉴스 기사들 중 'low_freq_categories'에 해당하는 부정적 샘플을 찾아 반환
    - 각 질문 뉴스에 대해 최대 4개의 부정적 샘플을 선택

    변경 사항:
    - 사용자 history category 빈도를 기준으로 low_freq_categories를 필터링 조건으로 추가
    - 24시간 이내 뉴스 -> 조건 충족 안되면 더 과거의 뉴스
    """
    negative_ids = []
    negative_titles = []

    for news_id, click_time in zip(news_ids, click_times):
        time_window_start = click_time - pd.Timedelta(hours=24)

        # 1. 24시간 이내의 negative sample 후보 찾기
        candidate_news = news_df[
            (news_df['Publish'] >= time_window_start) & 
            (news_df['Publish'] <= click_time) & 
            (~news_df['News ID'].isin(used_ids)) & 
            (~news_df['Title'].isin(used_titles)) & 
            (news_df['SubCategory'].isin(low_freq_categories))
        ]

        # 제목 중복 제거
        candidate_news = candidate_news.drop_duplicates(subset='Title')

        needed = 4
        selected_news_list = []

        if len(candidate_news) >= needed:
            # 24시간 이내에서 충분히 구할 수 있는 경우
            selected_news = candidate_news.sample(n=needed, random_state=42)
            selected_news_list.append(selected_news)
        else:
            # 24시간 이내에서 부족한 경우
            selected_news_list.append(candidate_news)
            needed -= len(candidate_news)

            # 2. 더 과거에서 추가 검색 (가장 최신 순)
            extra_news = news_df[
                (news_df['Publish'] < time_window_start) &
                (~news_df['News ID'].isin(used_ids)) &
                (~news_df['Title'].isin(used_titles)) &
                (news_df['SubCategory'].isin(low_freq_categories))
            ]
            extra_news = extra_news.drop_duplicates(subset='Title')

            # time_window_start와의 시간 차이를 계산하여 가장 가까운 뉴스부터 선택
            extra_news = extra_news.assign(TimeDiff=(time_window_start - extra_news['Publish']).abs())
            extra_news = extra_news.sort_values(by='TimeDiff')

            extra_selected_news = extra_news.head(needed)
            selected_news_list.append(extra_selected_news)

        # 선택된 뉴스들 합치기
        selected_news = pd.concat(selected_news_list)

        # 최종적으로 제목 중복 제거 및 섞기
        selected_news = selected_news.drop_duplicates(subset='Title').sample(frac=1, random_state=42)

        # negative sample을 최대 4개만 사용
        selected_news = selected_news.head(4)

        similar_ids = selected_news['News ID'].tolist()
        similar_titles = selected_news['Title'].tolist()

        negative_ids.append(similar_ids)
        negative_titles.append(similar_titles)

        # 사용한 ID, Title 업데이트
        used_ids.update(similar_ids)
        used_titles.update(similar_titles)

    return negative_ids, negative_titles

def find_similar_question_news(news_df, news_ids, click_times, used_ids, used_titles):
    """
    목적: 
    - 질문 뉴스와 유사한 뉴스 기사들 중 부정적 샘플을 찾아 반환
    - 각 질문 뉴스에 대해 최대 4개의 부정적 샘플을 선택

    설명:
    - 각 질문 뉴스에 대해 클릭 시간 기준으로 24시간 이내에 발행된 뉴스 중에서 사용되지 않은 뉴스 ID와 제목을 가진 후보들을 찾음
    - 후보들 중에서 중복된 제목을 제거한 후, 필요한 수만큼 랜덤하게 샘플링
    - 필요한 수만큼의 샘플이 부족할 경우, 더 과거의 뉴스에서 추가로 샘플을 선택
    - 선택된 뉴스 ID와 제목을 리스트에 저장하고, 이후 중복 방지를 위해 used_ids와 used_titles를 업데이트
    """
    negative_ids = []  # negative sample의 뉴스 ID 리스트
    negative_titles = []  # negative sample의 뉴스 제목 리스트

    for id, click_time in zip(news_ids, click_times):
        time_window_start = click_time - pd.Timedelta(hours=24)
        candidate_news = news_df[
            (news_df['Publish'] >= time_window_start) & 
            (news_df['Publish'] <= click_time) & 
            (~news_df['News ID'].isin(used_ids)) & 
            (~news_df['Title'].isin(used_titles))
        ]

        # 제목별로 중복 제거
        candidate_news = candidate_news.drop_duplicates(subset='Title')

        # 필요한 negative sample 수
        needed = 4
        selected_news_list = []

        if len(candidate_news) >= needed:
            selected_news = candidate_news.sample(n=needed, random_state=42)  # 랜덤 sampling
            selected_news_list.append(selected_news)
        else:
            selected_news_list.append(candidate_news)
            needed -= len(candidate_news)

            # 추가로 필요한 negative sample을 더 먼 과거에서 가져오기
            extra_news = news_df[
                (news_df['Publish'] < time_window_start) &
                (~news_df['News ID'].isin(used_ids)) &
                (~news_df['Title'].isin(used_titles))
            ]
            extra_news = extra_news.drop_duplicates(subset='Title')
            extra_news = extra_news.assign(TimeDiff=(time_window_start - extra_news['Publish']).abs())
            extra_news = extra_news.sort_values(by='TimeDiff')
            extra_selected_news = extra_news.head(needed)
            selected_news_list.append(extra_selected_news)

        # 선택된 뉴스들을 하나의 DataFrame으로 결합
        selected_news = pd.concat(selected_news_list)

        # 최종적으로 제목 중복 제거 및 섞기
        selected_news = selected_news.drop_duplicates(subset='Title')
        selected_news = selected_news.sample(frac=1, random_state=42)

        # 만약 negative sample 수가 부족하면 가능한 만큼만 사용
        selected_news = selected_news.head(4)

        similar_ids = selected_news['News ID'].tolist()
        similar_titles = selected_news['Title'].tolist()

        negative_ids.append(similar_ids)
        negative_titles.append(similar_titles)

        # 사용된 ID, Title 업데이트
        used_ids.update(similar_ids)
        used_titles.update(similar_titles)

    return negative_ids, negative_titles

def save_user_file(user_data, output_folder_path, purpose):
    """
    목적: 
    - 사용자별 prompt data를 텍스트 파일로 저장

    설명:
    - 지정된 output_folder_path 경로에 기존의 .txt 파일들을 삭제
    - 각 사용자에 대해 user_id를 파일명으로 하는 .txt 파일을 생성하고, 해당 사용자 데이터를 저장
    """
    # 기존 파일들 삭제
    if os.path.exists(output_folder_path):
        for file_path in glob.glob(os.path.join(output_folder_path, "*.txt")):
            os.remove(file_path)
        print(f'[{purpose}] 기존 User Prompts 삭제')    

    os.makedirs(output_folder_path, exist_ok=True)  # 폴더 생성

    # user prompt를 각 user별 txt파일로 저장
    for user_id, content in user_data.items():
        file_path = os.path.join(output_folder_path, f"{user_id}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)

def save_metadata_file(meta_folder_path, token_counts_and_outputs, user_metadata, purpose):
    """
    목적: 
    - 메타데이터(토큰 수, 출력 수 등)를 텍스트 파일로 저장

    설명:
    - 지정된 meta_folder_path 경로에 기존의 .txt 파일들을 삭제
    - 각 사용자에 대해 토큰 수, 출력 수, 히스토리 수, 질문 수 등을 기록하여 output_metadata.txt 파일에 저장
    - 총 입력 토큰 수와 출력 토큰 수도 함께 기록
    """
    # 기존 파일들 삭제
    if os.path.exists(meta_folder_path):
        for file_path in glob.glob(os.path.join(meta_folder_path, "*.txt")):
            os.remove(file_path)    

    os.makedirs(meta_folder_path, exist_ok=True)    # 폴더 생성

    # metadata 저장 위치
    meta_file_path = os.path.join(meta_folder_path, "output_metadata.txt")

    total_input_tokens = 0  # 총 입력 토큰 수
    total_output_count = 0  # 총 출력 토큰 수

    # user별 metadata(token 수, history 수, question 수 등) 기록
    with open(meta_file_path, "w", encoding="utf-8") as meta_file:
        for user_id, token_count, output_count, num_news_ids, num_questions in sorted(token_counts_and_outputs, key=lambda x: int(x[0][1:])):
            output_line = (f"User ID: {user_id:<5} Input Tokens: {token_count:<6} Output Tokens: {output_count:<4}  "
                           f"History 수: {num_news_ids:<3}  Question 수: {num_questions}")
            total_input_tokens += token_count
            total_output_count += output_count

            meta_file.write(output_line + "\n")

        # 총 합계 기록
        total_line = f"\nTotal Input Tokens: {total_input_tokens}\nTotal Output Tokens: {total_output_count}"
        print(f"[{purpose}] {total_line}\n")
        meta_file.write(total_line + "\n")

def save_hidden_positions(meta_folder_path, hidden_positions_data):
    """
    목적: 
    - 질문의 정답 위치(인덱스)를 텍스트 파일로 저장

    설명:
    - 각 사용자에 대해 정답 위치를 기록하여 hidden_positions.txt 파일에 저장
    """
    hidden_positions_file_path = os.path.join(meta_folder_path, "hidden_positions.txt")
    with open(hidden_positions_file_path, "w", encoding="utf-8") as hidden_file:
        for user_id, positions in hidden_positions_data:
            hidden_file.write(f"{user_id:<5}: {positions}\n")

def generate_demonstrations(user_name, history_titles, history_negative_titles):
    """
    목적: 
    - Demo용 문장을 생성하여 사용자 프롬프트에 추가

    설명:
    - 사용자의 히스토리 제목과 부정적 샘플 제목을 기반으로 데모 문장을 생성
    - 실제 정답의 위치를 [MASK]로 표시하고, 후보 제목들을 무작위로 섞어 표시
    """
    demonstration_lines = []
    # demonstration_lines.append(f"Here are some demonstrations intended for {user_name}.\n")

    for i, (h_title, neg_titles) in enumerate(zip(history_titles, history_negative_titles)):
        candidates = neg_titles + [h_title]
        random.shuffle(candidates)  # 후보 제목들 섞기
        index_of_actual = candidates.index(h_title) + 1  # 실제 정답의 인덱스 (1부터 시작)

        demonstration_lines.append(f"{user_name} prefers most [MASK] among the following five articles:")
        for j, candidate_title in enumerate(candidates):
            demonstration_lines.append(f"{j+1}: {candidate_title}")
        demonstration_lines.append(f"The index number of the [MASK] is {index_of_actual}.\n")

    return demonstration_lines

def create_prompts(purpose, model, user_count, max_question, max_history=300, save_folder="user_prompts", start_count=0):
    """
    목적: 
    - 전체 프로세스를 통해 사용자별 프롬프트를 생성하고, 관련 파일들을 저장

    설명:
    - 사용자 데이터를 순회하면서 히스토리 뉴스와 질문 뉴스의 ID 및 클릭 시간을 추출
    - positive 및 negative sample을 찾아 프롬프트를 생성
    - 생성된 프롬프트와 metadata를 파일로 저장
    - purpose에 따라 프롬프트의 내용이 달라지며, with_negative인 경우 부정적 샘플을 포함
    """

    now = datetime.datetime.now()
    formatted_date = now.strftime("%y%m%d")

    save_folder = f'../../prompts/[{formatted_date}-2] {save_folder}'

    # 사용자별 데이터 초기화
    user_data = {}
    token_counts_and_outputs = []
    hidden_positions_data = []

    # 히스토리의 최대 길이 설정
    max_history = int(f'-{max_history}')

    # tiktoken을 사용하여 토큰 수 계산
    encoding = tiktoken.encoding_for_model(model)

    # 사용자 목록 생성 (데이터프레임 'user'의 상위 user_count만 선택)
    selected_users = user.iloc[start_count:user_count]

    for idx, row in selected_users.iterrows():
        user_id = row['User']
        user_name = f"User #{user_id[0:]}"  # 사용자 이름 형식 지정

        # history 뉴스 ID 및 클릭 시간 추출 (중복 제거됨)
        history_str = row['History']
        history_news_ids, history_click_times = extract_news_ids_and_click_times(history_str)
        history_news_ids = history_news_ids[max_history:]
        history_click_times = history_click_times[max_history:]

        # history 뉴스 Title 추출
        history_titles = []
        for news_id in history_news_ids:
            matching_rows = history_news[history_news['News ID'] == news_id]
            if not matching_rows.empty:
                title = matching_rows.iloc[0]['Title']
                history_titles.append(title)

        # question 뉴스 ID 및 클릭 시간 추출 (중복 제거됨)
        question_str = row['Question']
        question_news_ids, question_click_times = extract_news_ids_and_click_times(question_str)

        # question 뉴스 제목 추출
        question_titles_list = []
        for news_id in question_news_ids:
            matching_rows = question_news[question_news['News ID'] == news_id]
            if not matching_rows.empty:
                title = matching_rows.iloc[0]['Title']
                question_titles_list.append(title)
                
                

        # used_ids와 used_titles 초기화 (이미 사용된 ID와 제목)
        used_ids = set(history_news_ids + question_news_ids)
        used_titles = set(history_titles + question_titles_list)
        

        user_history_categories = history_news[history_news['News ID'].isin(history_news_ids)]['SubCategory']
        category_counts = Counter(user_history_categories)
        all_categories = set(history_news['SubCategory'].unique()).union(question_news['SubCategory'].unique())
        low_freq_categories = {cat for cat in all_categories if category_counts[cat] <= 0}

        # history 뉴스에 대한 negative 샘플 찾기
        if purpose == 'with_negative':
            history_negative_ids, history_negative_titles = find_similar_news(
                history_news, history_news_ids, history_click_times, used_ids, used_titles, low_freq_categories
            )

            # 데모 생성
            demonstration_lines = generate_demonstrations(
                user_name, history_titles, history_negative_titles
            )
        else:
            demonstration_lines = []
            demonstration_lines.append("Here are some demonstrations")
            demonstration_lines.append(instruction)

        # question 뉴스에 대한 negative sample 찾기
        negative_question_ids, negative_question_titles = find_similar_question_news(
            question_news, question_news_ids, question_click_times, used_ids, used_titles
        )

        # 각 question에 대한 candidate title 준비
        all_candidate_titles = []
        hidden_positions = []

        for q_title, neg_titles in zip(question_titles_list, negative_question_titles):
            
            neg_titles = [title for title in neg_titles if title != q_title]    # neg_titles에서 q_title과 동일한 제목을 제거
            neg_titles = list(dict.fromkeys(neg_titles))    # neg_titles에서 제목의 중복 제거

            # negative 샘플이 4개 미만일 경우 처리  (현재는 반복을 중단)
            while len(neg_titles) < 4:
                break  # 일단 반복을 중단하고 현재 상태로 진행

            # 최대 4개의 negative 샘플과 q_title을 후보로 사용
            candidates = neg_titles[:4] + [q_title]
            # 후보에서 제목의 중복 제거
            candidates = list(dict.fromkeys(candidates))

            # 후보가 5개가 되도록 보장 (필요 시 추가 로직 구현 가능)
            if len(candidates) < 5:
                print(f'idx, question수 5개 미만')
                pass

            random.shuffle(candidates)  # 후보 섞기
            all_candidate_titles.append(candidates)
            index_of_actual = candidates.index(q_title) + 1  # 실제 정답의 인덱스 (1부터 시작)
            hidden_positions.append(index_of_actual)


        # 프롬프트 텍스트 생성
        prompt_lines = []

        # 데모 부분 추가
        prompt_lines.extend(demonstration_lines)

        # 사용자의 클릭 히스토리 추가
        if purpose == 'only_positive':
            prompt_lines.append(f"\nThe news articles that {user_name} clicked before are as follows:")
            for i, title in enumerate(history_titles):
                prompt_lines.append(f"{i+1}. {title}")
            prompt_lines.append("\n")

        # question 부분 추가
        if purpose == 'with_negative':
            # prompt_lines.append(f"Based on these demonstrations for {user_name}, predict the index number of news article that best fits in the position labeled [MASK] for each following question in terms of {user_name}.\n")
            prompt_lines.append(f"Based on {user_name}'s preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.\n")
        # else:
        #     prompt_lines.append(f"Based on the articles the user clicked before, {user_name} prefers most [MASK] among the following five articles:")

        for i, candidates in enumerate(all_candidate_titles[:max_question]):
            if purpose == 'with_negative':
                prompt_lines.append(f"{user_name} prefers most [MASK] among the following five articles:")
            else:
                prompt_lines.append(f"Based on the articles the user clicked before, {user_name} prefers most [MASK] among the following five articles:")
            for j, candidate_title in enumerate(candidates):
                prompt_lines.append(f"{j+1}: {candidate_title}")
            prompt_lines.append(f"Question {i+1}. The index number of the [MASK] is ?\n")
            num_questions = i+1

        prompt_lines.append(f"Please provide just the answers to each of {user_name}'s question without any explanations.")
        prompt_text = '\n'.join(prompt_lines)
        
        user_data[user_id] = prompt_text
        hidden_positions_data.append((user_id, hidden_positions))
        token_count = len(encoding.encode(prompt_text))  # 토큰 수 계산
        output_count = num_questions  # 출력 수
        num_history_titles = len(history_titles)  # 히스토리 제목 수
        token_counts_and_outputs.append((user_id, token_count, output_count, num_history_titles, num_questions))

    # prompt 및 metadata 저장
    output_folder_path = os.path.join(save_folder, purpose)
    save_user_file(user_data, output_folder_path, purpose)

    # metadata 파일 저장
    meta_folder_path = os.path.join(save_folder, purpose, 'metadata')
    save_metadata_file(meta_folder_path, token_counts_and_outputs, None, purpose)
    save_hidden_positions(meta_folder_path, hidden_positions_data)


In [ ]:
create_prompts(purpose='only_positive',
               model="gpt-4o-mini",
               user_count=1000,
               max_question=15,
               max_history=60,
               save_folder="임시용2",
               start_count=0
               )

create_prompts(purpose='with_negative',
               model="gpt-4o-mini",
               user_count=1000,
               max_question=15,
               max_history=60,
               save_folder="임시용2",
               start_count=0
               )

[with_negative] 
Total Input Tokens: 226283
Total Output Tokens: 280



### example

In [24]:
user = pd.read_csv('data/user.tsv', sep='\t', names=['User', 'History', 'Train', 'Question'])
user['History'] = user['History'] + user['Train']
user = user.drop(columns=['Train'])

user["History"] = user["History"].apply(lambda x: " ".join([item.split(",")[0] for item in x.split(";") if item]))
user["Question"] = user["Question"].apply(lambda x: " ".join([item.split(",")[0] for item in x.split(";") if item]))

user

,User,History,Question
0,U1,N1 N1 N317 N329 N521 N533 N874 N830 N868 N933 ...,N4296
1,U2,N1 N1 N3 N98 N1 N329 N15 N385 N385 N817 N831 N...,N3589 N3553 N3987 N4395 N4453
2,U3,N1 N1 N387 N385 N329 N335 N496 N868 N1057 N830...,N4296
3,U4,N2 N2 N148 N206 N148 N335 N329 N408 N868 N879 ...,N4475
4,U5,N3 N3 N3 N230 N335 N329 N329 N426 N408 N432 N7...,N3539 N3550 N3550 N3615 N3958 N3615 N3553 N4296
...,...,...,...
21934,U21935,N1412 N1412 N2252 N2168 N1845 N2831 N2730 N209...,N3890 N3701
21935,U21936,N1448 N1448 N1519 N1536 N1519 N1517 N1845 N181...,N4274 N4053 N4296
21936,U21937,N1379 N1379 N2035 N2071 N2451 N2514 N2252 N288...,N4125 N4053 N3862 N4274 N4375 N4549 N4555 N429...
21937,U21938,N1504 N1504 N1448 N1528 N1604 N1528 N1993 N216...,N3556 N4107 N3589 N3589 N3589 N4470 N4350 N443...


In [ ]:
history_news[history_news['News ID'].isin(history_news_ids)]['Category']